The first set of code cells need to be run first as they contain the module imports and the functions used in the script. Then run the script, remembering to update the file_to_process value before doing so.

Load document.

In [1]:
import os
import pickle
import re

def retrieve_raw_data(file_name):
    os.chdir('/media/nathan/FLASH DRIVE/Hmong corpus material/medical_corpus')
    f = open(file_name, 'r')
    doc_data = f.readlines()
    f.close()
    return doc_data

Every time there is a period, a question mark, or other separating punctuation, this should be split into a separate sentence, and a sentence list needs to be populated.

In [2]:
def extract_tagged_words(raw_data):
    sentences = []
    current_sentence = []
    for line in raw_data:
        for word in line.strip().split(' '):
            if word[:3] == '\//':
                # set special temporary replacement for / character
                temp_word = '@@@/' + word[3:]
            if word[:2] == '//':
                temp_word = '@@@/' + word[2:]
            else:
                temp_word = word
            processed_word = temp_word.split('/')
            if word in ['?/O-PU', './O-PU']:
                current_sentence.append(processed_word)
                sentences.append(current_sentence)
                current_sentence = []
            # restore proper character after split
            elif processed_word[0] == '@@@':
                processed_word[0] = '/'
                current_sentence.append(processed_word)
            else:
                current_sentence.append(processed_word)
        # if line ends in EOL rather than a question mark or period
        if current_sentence != []:
            sentences.append(current_sentence)
            current_sentence = []
    return sentences

Check data for proper format.

In [3]:
def check_format(data_in):
    errors = []
    for i, sent in enumerate(data_in):
        for j, word in enumerate(sent):
            if len(word) != 2:
                errors.append((word, i, j))
                continue
            out = word[1].split('-')
            if len(out) != 2:
                errors.append((word, i, j))
                continue
            if len(out[1]) != 2:
                errors.append((word, i, j))
                continue
            if out[0] not in ['B', 'I', 'O']:
                errors.append((word, i, j))
                continue
    return errors

Process data and pickle the result so that it can be opened in the Medical_Corpus_Database file for further scripting.

In [4]:
file_to_process = '11.txt'
doc_data = retrieve_raw_data(file_to_process)
tagged_words = extract_tagged_words(doc_data)
print(check_format(tagged_words))
if len(check_format(tagged_words)) == 0:
    os.chdir(os.path.expanduser('~/python_workspace/medical_corpus_scripting/pickling'))
    f = open(re.sub('\.', '_', file_to_process) + '.pkl', 'wb')
    pickle.dump(tagged_words, f)
    f.close()
else:
    raise TypeError

[]
